In [1]:
import torch
from torch import nn
import kornia
import best.singular_point as sp
from teste_util import *

# Fixar a semente do Torch para operações específicas
fixed_seed()

# leitura dos dados
# trainloader,testloader =read_dataload_flower(sp.args.img_size,'./data/datasets',batch_size=60)
# trainloader,testloader =read_dataload_fibers(sp.args.img_size)
trainloader,testloader=read_dataload_woods(sp.args.img_size)
iterator=iter(testloader)
img,labels = next(iterator)
print(img.shape)

# Carregar o modelo singular points
# path_siamese = './data/models/sp2_mp_25.pth'
# path_siamese = './data/models/sp2_85.pth'
path_siamese = './data/models/sp_map_fo_30.pth'

sp.args.num_channels = 1
model = sp.SingularPoints(args=sp.args).to(sp.device)
load_model(model,path_siamese,sp.device)

#gerar variacao de transformacoes pespectivas e fotometrica
params_lists =AugmentationParamsGenerator(6,img.shape)


train True   213
train False   496
496
torch.Size([31, 1, 120, 120])
Namespace(exported=False, num_channels=1, pyramid_levels=3, scale_pyramid=1.3, dim_first=3, dim_second=5, dim_third=8, group_size=36, epochs=70, border_size=12, box_size=21, nms_size=5, img_size=120, batch_size=16, path_data='./data', path_model='model.pt', is_loss_ssim=True, margin_loss=2.0, outlier_rejection=False, show_feature=False)


/home/wagner/.local/lib/python3.11/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)


Model loaded from ./data/models/sp_map_fo_30.pth


In [2]:

from kornia.feature.scale_space_detector import get_default_detector_config, MultiResolutionDetector
from kornia.feature.keynet import KeyNetDetector

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


keynet_default_config = {
    'num_filters': 8,
    'num_levels': 3,
    'kernel_size': 5,
    # 'Detector_conf': {'nms_size': 15, 'pyramid_levels': 1, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 22.0},
    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 0, 'up_levels': 0, 'scale_factor_levels': 1.0, 's_mult': 5.0},
   #  'Detector_conf': {'nms_size': 5, 'pyramid_levels': 3, 'up_levels': 2, 'scale_factor_levels': 1.3, 's_mult': 12.0},# flowers 420/1020  0.411
   #  'Detector_conf': {'nms_size': 7, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 14.0}#fibers
   #  'Detector_conf': {'nms_size': 7, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 14.0}#woods 253/496 - 0.510
#    'Detector_conf': {'nms_size': 9, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.5, 's_mult': 14.0},#woods  hard 262/496 - 0.528
   # 'Detector_conf': {'nms_size': 7, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 14.0},#woods  sift 245/496 - 0.494
}
kornia.feature.SIFTDescriptor(32, rootsift=True)
detector1 = KeyNetDetector(num_features=60, keynet_conf=keynet_default_config,ori_module=kornia.feature.LAFOrienter(32)).to(device)

In [3]:
# class Detector(nn.Module):
#         def __init__(self,model) -> None:
#            super().__init__()
#            self.model = model
#         def forward(self, x):
#             features_key,features_key_summary,features_ori,features_ori_summary,max_coords_values, max_map= self.model(x)
#             return features_key_summary
# detec = Detector(model)
# detector1 = CustomNetDetector(detec,PS=32,keynet_conf=keynet_default_config,num_features=60).to(sp.device)

In [4]:
# import argparse

# ## for fix seed
# import random, torch, numpy 
# def get_config(jupyter=False):
#     parser = argparse.ArgumentParser(description='Train REKD Architecture')

#     ## basic configuration
#     parser.add_argument('--data_dir', type=str, default='../ImageNet2012/ILSVRC2012_img_val', #default='path-to-ImageNet',
#                             help='The root path to the data from which the synthetic dataset will be created.')
#     parser.add_argument('--synth_dir', type=str, default='', 
#                             help='The path to save the generated sythetic image pairs.')
#     parser.add_argument('--log_dir', type=str, default='trained_models/weights',
#                             help='The path to save the REKD weights.')
#     parser.add_argument('--load_dir', type=str, default='',
#                         help='Set saved model parameters if resume training is desired.')                            
#     parser.add_argument('--exp_name', type=str, default='REKD',
#                             help='The Rotaton-equivaraiant Keypoint Detection (REKD) experiment name')
#     ## network architecture
#     parser.add_argument('--factor_scaling_pyramid', type=float, default=1.2,
#                         help='The scale factor between the multi-scale pyramid levels in the architecture.')
#     parser.add_argument('--group_size', type=int, default=36,  
#                         help='The number of groups for the group convolution.')
#     parser.add_argument('--dim_first', type=int, default=2,
#                         help='The number of channels of the first layer')
#     parser.add_argument('--dim_second', type=int, default=2,
#                         help='The number of channels of the second layer')
#     parser.add_argument('--dim_third', type=int, default=2,
#                         help='The number of channels of the thrid layer')                       
#     ## network training
#     parser.add_argument('--batch_size', type=int, default=16,
#                         help='The batch size for training.')
#     parser.add_argument('--num_epochs', type=int, default=20, 
#                         help='Number of epochs for training.')
#     ## Loss function  
#     parser.add_argument('--init_initial_learning_rate', type=float, default=1e-3,
#                         help='The init initial learning rate value.')
#     parser.add_argument('--MSIP_sizes', type=str, default="8,16,24,32,40",
#                         help='MSIP sizes.')
#     parser.add_argument('--MSIP_factor_loss', type=str, default="256.0,64.0,16.0,4.0,1.0",
#                         help='MSIP loss balancing parameters.')
#     parser.add_argument('--ori_loss_balance', type=float, default=100., 
#                         help='')
#     ## Dataset generation
#     parser.add_argument('--patch_size', type=int, default=192,
#                         help='The patch size of the generated dataset.')
#     parser.add_argument('--max_angle', type=int, default=180,
#                         help='The max angle value for generating a synthetic view to train REKD.')
#     parser.add_argument('--min_scale', type=float, default=1.0,
#                         help='The min scale value for generating a synthetic view to train REKD.')
#     parser.add_argument('--max_scale', type=float, default=1.0,
#                         help='The max scale value for generating a synthetic view to train REKD.')
#     parser.add_argument('--max_shearing', type=float, default=0.0,
#                         help='The max shearing value for generating a synthetic view to train REKD.')
#     parser.add_argument('--num_training_data', type=int, default=9000,
#                         help='The number of the generated dataset.')
#     parser.add_argument('--is_debugging', type=bool, default=False,
#                         help='Set variable to True if you desire to train network on a smaller dataset.')
#     ## For eval/inference
#     parser.add_argument('--num_points', type=int, default=1500,
#                         help='the number of points at evaluation time.')
#     parser.add_argument('--pyramid_levels', type=int, default=5,
#                         help='downsampling pyramid levels.')
#     parser.add_argument('--upsampled_levels', type=int, default=2,
#                         help='upsampling image levels.')
#     parser.add_argument('--nms_size', type=int, default=15,
#                         help='The NMS size for computing the validation repeatability.')
#     parser.add_argument('--border_size', type=int, default=15,
#                         help='The number of pixels to remove from the borders to compute the repeatability.')
#     ## For HPatches evaluation
#     parser.add_argument('--hpatches_path', type=str, default='./datasets/hpatches-sequences-release',
#                         help='dataset ')
#     parser.add_argument('--eval_split', type=str, default='debug',
#                         help='debug, view, illum, full, debug_view, debug_illum ...')      
#     parser.add_argument('--descriptor', type=str, default="hardnet",
#                         help='hardnet, sosnet, hynet')    

#     args = parser.parse_args() if not jupyter else parser.parse_args(args=[])   

#     fixed_seed()

#     if args.synth_dir == "":
#         args.synth_dir = 'datasets/synth_data'

#     args.MSIP_sizes = [int(i) for i in args.MSIP_sizes.split(",")]
#     args.MSIP_factor_loss =[float(i) for i in args.MSIP_factor_loss.split(",")]

#     return args


# from external.REKD import REKD, count_model_parameters
# from external.hardnet_pytorch import HardNet

# args = get_config(jupyter=True)
# args.load_dir =  'trained_models/release_group36_f2_s2_t2.log/best_model.pt'

# from torch import nn
# class Detector(nn.Module):
#         def __init__(self) -> None:
#            super().__init__()
#            detec = REKD(args, device)
#            detec = detec.to(device) ## use GPU
#            detec.load_state_dict(torch.load(args.load_dir,map_location=device))  ## Load the PyTorch learnable model parameters.
#            detec.eval()
#            self.model = detec
           
#         def forward(self, x):
#             features_key,features_ori= self.model(x)
#             return features_key


# detec = Detector()
# detector1 = CustomNetDetector(detec,PS=32,keynet_conf=keynet_default_config,num_features=60).to(sp.device)

In [5]:
import numpy as np
from scipy.spatial.distance import cdist
import kornia.feature as KF
import matplotlib.pyplot as plt
import kornia as K

def visualize_LAF(img, LAF, img_idx = 0):
    x, y = KF.laf.get_laf_pts_to_draw(LAF, img_idx)
    print(x[0][:5],y[0][:5])
    plt.figure()
    plt.imshow(K.utils.tensor_to_image(img[img_idx]))
    plt.plot(x, y, 'r')
    plt.show()
    return

import matplotlib.pyplot as plt
import numpy as np

def visualize_matching_images(img1, LAF1, img2, LAF2, matches):
    print(LAF1.shape,LAF2.shape)
    x1, y1 = KF.laf.get_laf_pts_to_draw(LAF1, 0)  # Pontos da primeira imagem
    x2, y2 = KF.laf.get_laf_pts_to_draw(LAF2, 0)  # Pontos da segunda imagem

    # Converte as listas x2 e y2 em arrays NumPy
    x2 = np.array(x2)
    y2 = np.array(y2)

    # Crie uma imagem combinada concatenando as duas imagens lado a lado
    combined_image = np.concatenate((K.utils.tensor_to_image(img1), K.utils.tensor_to_image(img2)), axis=1)

    fig, ax = plt.subplots(1, 1, figsize=(12, 6))  # Cria uma figura com uma subplot

    # Plota a imagem combinada
    ax.imshow(combined_image)
    ax.axis('off')
    # Plota os pontos correspondentes nas duas imagens
    ax.plot(x1, y1, 'c')  # 'ro' representa pontos vermelhos na primeira imagem
    ax.plot(x2 + img1.shape[1], y2, 'y')  # Desloca os pontos azuis na segunda imagem para a direita

    points1 =kornia.feature.get_laf_center(LAF1)[0].cpu()
    points2 =kornia.feature.get_laf_center(LAF2)[0].cpu()
    print(points1.shape,points2.shape)
    for match in matches:
        x1_match, y1_match = points1[match[0],0], points1[match[0],1]
        x2_match, y2_match = points2[match[1],0] + img1.shape[1], points2[match[1],1]
       
        ax.plot([x1_match, x2_match], [y1_match, y2_match], '-', color='red', lw=1.5)

    plt.tight_layout()
    plt.show()

    
def plot_matches_keypoints(image1, keypoints1, image2, keypoints2, matches, **kwargs):
    print('image1 shape: ',image1.shape,image1.dtype,image2.shape,image2.dtype)
    # Concatenar as duas imagens lado a lado
    combined_image = np.concatenate((image1, image2), axis=1)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(combined_image)
    ax.axis('off')

    # Desenhar pontos correspondentes e linhas conectando-os
    offset = image1.shape[1]

    for i, (x, y) in enumerate(keypoints1):
        ax.plot(x, y, 'o',markerfacecolor='none', markeredgecolor='r',
                markersize=20, markeredgewidth=1)
        ax.annotate(str(i), (x, y), color='r',xytext=(10, 10), textcoords='offset points', fontsize=12)

    for i, (x, y) in enumerate(keypoints2):
        ax.plot(x+offset, y, 'o',markerfacecolor='none', markeredgecolor='r',
                markersize=20, markeredgewidth=1)
        ax.annotate(str(i), (x+offset, y), color='r',xytext=(10, 10), textcoords='offset points', fontsize=12)

    for match in matches:
        x1, y1 = keypoints1[match[0],0], keypoints1[match[0],1]
        x2, y2 = keypoints2[match[1],0]+offset, keypoints2[match[1],1]
        ax.plot([x1, x2], [y1, y2], '-', color='lime', lw=0.5)

    plt.tight_layout()
    plt.show()

def plot_image_with_keypoints(image_tensor, keypoints_tensor):
    # Converter a imagem tensorial em objeto PIL.Image
    image = kornia.utils.tensor_to_image(image_tensor)
    # Plotar a imagem e os keypoints
    plt.imshow(image)
    if keypoints_tensor is not None:
        # Extrair as coordenadas x e y dos keypoints
        keypoints_x = keypoints_tensor[:,0].flatten().tolist()
        keypoints_y = keypoints_tensor[:,1].flatten().tolist()
        plt.scatter(keypoints_x, keypoints_y, c='red')
    plt.show()
    
def filtrar_keypoints(lista_de_pontos, tensor_mascara):
    # Verificar se as coordenadas estão dentro das dimensões
    dimensao_max_x, dimensao_max_y = tensor_mascara.shape[1] - 1, tensor_mascara.shape[0] - 1
    pontos_filtrados = [
        ponto.tolist()  for ponto in lista_de_pontos 
        if 0 <= ponto[0] <= dimensao_max_x 
        and 0 <= ponto[1] <= dimensao_max_y 
        and tensor_mascara[int(ponto[1]), int(ponto[0])] 
    ]
    return torch.tensor(pontos_filtrados)

def find_best_matching_indices_knn(points1, points2, threshold, k=3):
    if len(points1) == 0 or len(points2) == 0:
        return []
    distances = cdist(points1, points2)
    best_indices = np.argsort(distances, axis=1)[:, :k]
    best_distances = np.take_along_axis(distances, best_indices, axis=1)
    matched = []

    for i in range(len(points1)):
        min_distance = np.min(best_distances[i])        
        if min_distance < threshold:
            best_index = np.argmin(best_distances[i])
            matched.append((i, best_indices[i, best_index]))

    return matched


def detect_extract_feat_in_batch(batch1,aug_list, detector):
    origem_total = []
    matches_total = []
    with torch.no_grad():
        for img1  in batch1:            
            lafs1, resps1 = detector(img1[None])

            B,C,H,W = img1[None].shape
            mask = torch.ones(B,C,H,W).to(img1.device) 
            
            #lafs1 to points1
            points1 =kornia.feature.get_laf_center(lafs1)

            if( points1.shape[1] == 0):
                print('aug_list shape: ',points1.shape) 
                continue         
            img2,mask_t,ponts_t=aug_list(img1,mask,points1)            
            
            lafs2, resps2 = detector(img2)
            points2 =kornia.feature.get_laf_center(lafs2)     
            filtered_points1 = filtrar_keypoints(ponts_t[0],mask_t[0,0].bool())            
            filtered_points2 = filtrar_keypoints(points2[0],mask_t[0,0].bool())
            print("shape p & f",points1.shape,filtered_points1.shape,filtered_points2.shape)
            matches = find_best_matching_indices_knn(filtered_points1.cpu(), filtered_points2.cpu(), threshold=1.0, k=1)  
            matches2 = find_best_matching_indices_knn(ponts_t[0].cpu(), points2[0].cpu(), threshold=0.6, k=1)  
            
            # visualize_matching_images(img1[0], lafs1, img2[0,0], lafs2,matches2)
            print(img1.shape,img1.device,img2[0].shape,img2.device)
            # plot_matches_keypoints(img2[0,0].cpu(), filtered_points1, img2[0,0].cpu(), filtered_points2, matches)
            if( filtered_points1.shape[0] == 0 or filtered_points2.shape[0] == 0):
                print('filtered_points1 shape: ',filtered_points1.shape,'filtered_points2 shape: ',filtered_points2.shape)
                continue            
            origem_total.append(len(filtered_points1))
            matches_total.append(len(matches))            
    return (np.mean(matches_total)/np.mean(origem_total))*100



In [6]:
fixed_seed()
params_lists.aug_list.data_keys =["input"]

transforms = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.2, 0.2), scale=(0.95, 1.05), shear=10,p=0.8),
    kornia.augmentation.RandomPerspective(0.2, p=0.7),
    kornia.augmentation.RandomBoxBlur((4,4),p=0.5),
    # kornia.augmentation.RandomEqualize(p=0.3),
    data_keys=["input", "mask","keypoints"],
    same_on_batch=True,
    # random_apply=10,
)

acertos = 0
total = 0
from tqdm.notebook import tqdm
pbar =  tqdm(testloader)
matches_total = []
for imgs_batch,labels_batch in pbar:# itera em todo dataset
    imgs_batch = imgs_batch.to(device)
    mean = detect_extract_feat_in_batch(imgs_batch,transforms,detector1)
    matches_total.append(mean)
    print('match of batch ',mean)
print('match of dataset ',np.mean(matches_total))


  0%|          | 0/16 [00:00<?, ?it/s]

shape p & f torch.Size([1, 60, 2]) torch.Size([60, 2]) torch.Size([60, 2])
torch.Size([1, 120, 120]) cuda:0 torch.Size([1, 120, 120]) cuda:0
shape p & f torch.Size([1, 60, 2]) torch.Size([52, 2]) torch.Size([54, 2])
torch.Size([1, 120, 120]) cuda:0 torch.Size([1, 120, 120]) cuda:0
shape p & f torch.Size([1, 60, 2]) torch.Size([60, 2]) torch.Size([60, 2])
torch.Size([1, 120, 120]) cuda:0 torch.Size([1, 120, 120]) cuda:0
shape p & f torch.Size([1, 60, 2]) torch.Size([58, 2]) torch.Size([58, 2])
torch.Size([1, 120, 120]) cuda:0 torch.Size([1, 120, 120]) cuda:0
shape p & f torch.Size([1, 60, 2]) torch.Size([50, 2]) torch.Size([55, 2])
torch.Size([1, 120, 120]) cuda:0 torch.Size([1, 120, 120]) cuda:0
shape p & f torch.Size([1, 60, 2]) torch.Size([60, 2]) torch.Size([52, 2])
torch.Size([1, 120, 120]) cuda:0 torch.Size([1, 120, 120]) cuda:0
shape p & f torch.Size([1, 60, 2]) torch.Size([57, 2]) torch.Size([52, 2])
torch.Size([1, 120, 120]) cuda:0 torch.Size([1, 120, 120]) cuda:0
shape p & f t